In [1]:
library(tidyverse)
library(clusterProfiler)
library(WGCNA)
library(parallel)
library(pheatmap)
library(RColorBrewer)
library(ggplotify)
library(grid)

# Custom package
library(rutils)

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.0.6     v dplyr   1.0.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



clusterProfiler v3.18.0  For help: https://guangchuangyu.github.io/software/clusterProfiler

If you use clusterProfiler in published research, please cite:
Guangchuang Yu, Li-Gen Wang, Yanyan Han, Qing-Yu He. clusterProfiler: an R package for comparing biological themes among gene clusters. OMICS: A Journal of Integrative Biology. 2012, 16(5):284-287.


Attaching package: 'clusterProfiler'


The following object is masked from 'package:purrr':

    simplify


The following object is masked from 'package:stats':

    filter


L

In [2]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
projects <- c("cesc", "ucs", "ucec")
matrisome_path <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

event_code <- list("Alive" = 0, "Dead" = 1)

In [3]:
dset_idx <- 3
p_mm_thresh <- 0.05
q_me_thresh <- 0.05
save_results <- TRUE

In [4]:
network_mm_gs_df <- read_tsv(paste0(dirs$analysis_dir, "/network/", unified_dsets[dset_idx], "_gene_mm_gs.tsv"))
network_me_sig_df <- read_tsv(paste0(dirs$analysis_dir, "/network/", unified_dsets[dset_idx], "_eigengene_traits.tsv"))
demg_list <- read_lines(paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_demg_list.txt"))

lnames <- load(file = paste0(dirs$data_dir, "/saved_network_objects/", unified_dsets[dset_idx], "_tumor_data.RData"))
lnames <- load(file = paste0(dirs$data_dir, "/saved_network_objects/", unified_dsets[dset_idx], "_tumor_network.RData"))


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  .default = col_double(),
  geneID = col_character(),
  module = col_character()
)
i Use `spec()` for the full column specifications.



-- Column specification ------------------------------------------------------------------------------------------------
cols(
  module = col_character(),
  figo_stage_1_cor = col_double(),
  figo_stage_2_cor = col_double(),
  figo_stage_3_cor = col_double(),
  figo_stage_4_cor = col_double(),
  figo_stage_1_pval = col_double(),
  figo_stage_2_pval = col_double(),
  figo_stage_3_pval = col_double(),
  figo_stage_4_pval = col_double(),
  figo_stage_1_qval = col_double(),
  figo_stage_2_qval = col_double(),
  figo_stage_3_qval = col_double(),
  figo_stage_4_qval = col_double(),
  vital_hr = col_double(),
  vital_dev_cor = col_double(),
  vital_pval = col_double(),
  vital_qval = col_double()
)




In [5]:
# Subtract 1 from diagonal of TOM (so we don't count self-TOM)
tom_hm <- tom - diag(1, nrow(tom), nrow(tom))
tom_summary_df <- tibble(geneID = colnames(data_expr)) %>%
    mutate(tom_conn_sums = rowSums(tom_hm)) %>%
    mutate(tom_conn_means = rowMeans(tom_hm))
#     arrange(desc(tom_conn))
mean(tom_summary_df$tom_conn_sums)
quantile(tom_summary_df$tom_conn_sums)
mean(tom_summary_df$tom_conn_means)
quantile(tom_summary_df$tom_conn_means) %>% round(3)

[1] 3.942757

0%        25%        50%        75%       100% 
 0.3000765  1.7082870  2.6340958  4.4939226 26.5164136

[1] 0.00391535

0%   25%   50%   75%  100% 
0.000 0.002 0.003 0.004 0.026

In [6]:
wgcna_meta_ls <- wgcna_meta(network_me_sig_df, network_mm_gs_df, q_me_thresh, p_mm_thresh, colnames(data_expr))
wgcna_meta_ls[["demg_genes"]] <- wgcna_meta_ls$genes %>% intersect(demg_list)
wgcna_meta_ls[["n_sig_demg_genes"]] <- wgcna_meta_ls$demg_genes %>% length()

In [7]:
wgcna_meta_ls$modules

[1] "green"  "pink"   "salmon" "black"

In [8]:
for (m in wgcna_meta_ls$modules) {
    module_genes <- network_mm_gs_df %>%
        select(geneID, module) %>%
        filter(geneID %in% wgcna_meta_ls$demg_genes) %>%
        filter(module == m) %>%
        arrange(geneID) %>%
        pull(geneID)
    if (save_results) {
        module_genes %>% write_lines(paste0(dirs$analysis_dir, "/gene_lists_extra_network/", unified_dsets[dset_idx], "_", m, "_demg.txt"))
    }
}